# Q4 Data Mayhem: Conjuring insights from the data (9 Marks)
## Advanced Data Analysis and Prediction
**Description:** This section will involve investigating unusual data patterns, building predictive models, and reducing dimensionality to gain deeper insights.



# 4.a Outliers in Fare and Tips (4 Marks):
(a) Use box plots to identify outliers in fare and tip values.
(b) Interpretation: Discuss how these outliers might skew average calculations.
(c) Suggest methods for handling these outliers in predictive models (e.g., removal or adjustment).



# 4.b Predicting Fare (5 Marks):
(a) Implement a regression model using distance, trip duration, and location as predictors to estimate fares.
(b) Model Evaluation: Present metrics such as RMSE to assess model accuracy.
(c) Discuss the effectiveness of the model, noting any potential limitations or areas for improvement.